# 1. CARGA DE DATOS



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler #scala datos numericos
from sklearn.model_selection import train_test_split #entreno y  evaluo
from sklearn.tree import DecisionTreeClassifier #arbol decision
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, r2_score

In [ ]:
# Cargar y limpiar el datast
df = pd.read_csv("/content/drive/MyDrive/Bootcamp/BBDD/house-prices.csv")

# 2. EDA

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Home          128 non-null    int64 
 1   Price         128 non-null    int64 
 2   SqFt          128 non-null    int64 
 3   Bedrooms      128 non-null    int64 
 4   Bathrooms     128 non-null    int64 
 5   Offers        128 non-null    int64 
 6   Brick         128 non-null    object
 7   Neighborhood  128 non-null    object
dtypes: int64(6), object(2)
memory usage: 8.1+ KB


* En un primer análisis no se observan datos faltantes.

# Descripción de Columnas:
* Price: Precio de venta de la casa
* SqFt: Tamaño de la casa en pies cuadrados.
* Bedrooms: Número de habitaciones.
* Bathrooms: Número de baños.
* Offers: Ofertas
* Brick: Ladrillo
* Neighborhood: Barrio (Ubicación)


In [ ]:
df.head()

,Home,Price,SqFt,Bedrooms,Bathrooms,Offers,Brick,Neighborhood
0,1,114300,1790,2,2,2,No,East
1,2,114200,2030,4,2,3,No,East
2,3,114800,1740,3,2,1,No,East
3,4,94700,1980,3,2,3,No,East
4,5,119800,2130,3,3,3,No,East


In [ ]:
df.isna().sum()

,0
Home,0
Price,0
SqFt,0
Bedrooms,0
Bathrooms,0
Offers,0
Brick,0
Neighborhood,0


* No se obervan valores faltantes

In [ ]:
df.isnull().sum()

,0
Home,0
Price,0
SqFt,0
Bedrooms,0
Bathrooms,0
Offers,0
Brick,0
Neighborhood,0


* No se observan valores nulos

In [ ]:
df.duplicated().sum()

np.int64(0)

* No hay valores duplicados

In [ ]:
df.describe().T.round()

,count,mean,std,min,25%,50%,75%,max
Home,128.0,64.0,37.0,1.0,33.0,64.0,96.0,128.0
Price,128.0,130427.0,26869.0,69100.0,111325.0,125950.0,148250.0,211200.0
SqFt,128.0,2001.0,212.0,1450.0,1880.0,2000.0,2140.0,2590.0
Bedrooms,128.0,3.0,1.0,2.0,3.0,3.0,3.0,5.0
Bathrooms,128.0,2.0,1.0,2.0,2.0,2.0,3.0,4.0
Offers,128.0,3.0,1.0,1.0,2.0,3.0,3.0,6.0


* Chequeo que los datos estadísticos estén bien distribuidos, no se observan datos extraños.

In [ ]:
#Chequeo las variables categóricas
df["Neighborhood"].value_counts()

,count
Neighborhood,
East,45
North,44
West,39


In [ ]:
df["Brick"].value_counts()

,count
Brick,
No,86
Yes,42


* Se observa que la variable Brick tien dos opciones, por lo tanto la clasificamos como categórica binaria, por lo tanto puedo convertirla a número

In [ ]:
df["Brick"] = df["Brick"].map({"No": 0, "Yes": 1})

In [ ]:
df.info() #Compruebo

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Home          128 non-null    int64 
 1   Price         128 non-null    int64 
 2   SqFt          128 non-null    int64 
 3   Bedrooms      128 non-null    int64 
 4   Bathrooms     128 non-null    int64 
 5   Offers        128 non-null    int64 
 6   Brick         128 non-null    int64 
 7   Neighborhood  128 non-null    object
dtypes: int64(7), object(1)
memory usage: 8.1+ KB


* Luego transformo la columna Neighborhood, ya que cuenta con 3 categorías nominales no tienen un orden lógico.

In [ ]:
df = pd.get_dummies(df, columns=["Neighborhood"])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Home                128 non-null    int64
 1   Price               128 non-null    int64
 2   SqFt                128 non-null    int64
 3   Bedrooms            128 non-null    int64
 4   Bathrooms           128 non-null    int64
 5   Offers              128 non-null    int64
 6   Brick               128 non-null    int64
 7   Neighborhood_East   128 non-null    bool 
 8   Neighborhood_North  128 non-null    bool 
 9   Neighborhood_West   128 non-null    bool 
dtypes: bool(3), int64(7)
memory usage: 7.5 KB


* como las nuevas columnas quedaron expresadas en booleanos, las cambiaré a int64 para que quede todo mi Data Frame con el mismo tipo.

In [ ]:
df[["Neighborhood_East", "Neighborhood_North", "Neighborhood_West"]] = df[["Neighborhood_East", "Neighborhood_North", "Neighborhood_West"]].astype(int)


In [ ]:
df.info() # Chequeo si se aplicó el cambio.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Home                128 non-null    int64
 1   Price               128 non-null    int64
 2   SqFt                128 non-null    int64
 3   Bedrooms            128 non-null    int64
 4   Bathrooms           128 non-null    int64
 5   Offers              128 non-null    int64
 6   Brick               128 non-null    int64
 7   Neighborhood_East   128 non-null    int64
 8   Neighborhood_North  128 non-null    int64
 9   Neighborhood_West   128 non-null    int64
dtypes: int64(10)
memory usage: 10.1 KB


* Ya corroboré que está mi data limpia, sin nulos, sin duplicados y mis variables categóricas codificadas.
Al hacer este ultimo .info analizo que la columna Home no aporta datos relevantes de información, por lo tanto procedo a eliminarla.

In [ ]:
df = df.drop(columns=["Home"]) # Home sólo representa el ID de la casa y no me aporta información util para realizar mi predicción.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   Price               128 non-null    int64
 1   SqFt                128 non-null    int64
 2   Bedrooms            128 non-null    int64
 3   Bathrooms           128 non-null    int64
 4   Offers              128 non-null    int64
 5   Brick               128 non-null    int64
 6   Neighborhood_East   128 non-null    int64
 7   Neighborhood_North  128 non-null    int64
 8   Neighborhood_West   128 non-null    int64
dtypes: int64(9)
memory usage: 9.1 KB


* Ahora que sí está completo todo mi EDA procedo a la preparacion de datos.

# 3. PREPARACIÓN DE DATOS

* Divide los datos en un conjunto de entrenamiento (80%) y un conjunto de prueba (20%).

In [ ]:
#División de Features y Target
X = df.drop(columns=["Price"])
y = df["Price"]

In [ ]:
X.head()

,SqFt,Bedrooms,Bathrooms,Offers,Brick,Neighborhood_East,Neighborhood_North,Neighborhood_West
0,1790,2,2,2,0,1,0,0
1,2030,4,2,3,0,1,0,0
2,1740,3,2,1,0,1,0,0
3,1980,3,2,3,0,1,0,0
4,2130,3,3,3,0,1,0,0


In [ ]:
# Separo datos de entrenamiento y prueba con Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

* test_size = 0.2 nos indica 80% entrenamineto, 20% prueba que es lo solicitado en el CORE.
* Aplico random_state =42 según lo visto en clases para asegurarme de obtener siempre el mismo resultado.

# 4. ENTRENAMIENTO DEL MODELO
* Entrena un modelo de regresión lineal utilizando los datos de entrenamiento.

In [ ]:
df.columns

Index(['Price', 'SqFt', 'Bedrooms', 'Bathrooms', 'Offers', 'Brick',
       'Neighborhood_East', 'Neighborhood_North', 'Neighborhood_West'],
      dtype='object')

In [ ]:
# Separo datos de entrenamiento y prueba con Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo
modelo = LinearRegression()

# Entrenar el modelo de regresión lineal

modelo.fit(X_train, y_train)

LinearRegression()

# 5. EVALUACIÓN DEL MODELO
* Evalúa el modelo utilizando el conjunto de prueba. Calcula el error cuadrático medio (MSE) y el R-cuadrado (R²) del modelo.

In [ ]:
# Predecir con los datos de prueba
y_pred = modelo.predict(X_test)

# Calcular MSE y R²
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
# # Imprimir los resultados
print(f"Error promedio: ${mse**0.5:,.0f}")
print(f"R² (qué tan bueno es mi modelo): {r2:.4f}")

Error promedio: $10,685
R² (qué tan bueno es mi modelo): 0.8063


# 6. PREDICCIÓN
* Usa el modelo entrenado para predecir el precio de una casa con las siguientes características:
Tamaño: 2000 pies cuadrados
Habitaciones: 3
Baños: 2

In [ ]:
# Creo un nuevo DataFrame con las características solicitadas
Prediccion_casa = pd.DataFrame([{
    "SqFt": 2000,
    "Bedrooms": 3,
    "Bathrooms": 2,

# Completo información faltante según mis elecciones:
    "Offers": 1,
    "Brick": 1,  # asumo que la casa sí tiene ladrillo.
    "Neighborhood_East": 0,
    "Neighborhood_North": 1, # Elegí que la casa esté ubicada en el barrio Norte.
    "Neighborhood_West": 0
    }])

# Predecir el precio con mi modelo entrenado
prediccion_precio = modelo.predict(Prediccion_casa)

print(f"Precio estimado: ${prediccion_precio[0]:,.0f}")

Precio estimado: $146,366


# CONCLUSIONES:


* El error Promedio de mi modelo es de $10.685 USD

---


* Mi modelo tiene una precisión de alrededor un 81%.


---

* La predicción del valor de la casa solicitada en el ultimo punto del Core, luego de aplicar mi modelo es de $146.366 USD